In [37]:
 pip install -Uq langgraph langsmith langchain_groq langchain_tavily grandalf

In [6]:
from os import environ
from google.colab import userdata

environ["GROQ_API_KEY"] = userdata.get('GROQ_API_KEY')
environ["TAVILY_API_KEY"] = userdata.get('TAVILY_API_KEY')
environ["LANGSMITH_API_KEY"] = userdata.get('LANGSMITH_API_KEY')

environ["LANGCHAIN_TRACING_V2"] = "true"
environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
environ["LANGCHAIN_PROJECT"] = "12-LangGraph-Add-Conversation-Summary"

In [124]:
from typing import TypedDict, Annotated
from langchain_groq import ChatGroq
from langchain_tavily import TavilySearch
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage, ChatMessage, RemoveMessage, ToolMessage
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import InMemorySaver
from langgraph.graph import StateGraph, START, END
from langchain_core.runnables import RunnableConfig


class State(TypedDict):
  messages: Annotated[list[ChatMessage], add_messages]
  summary: Annotated[str, "summary"]


search = TavilySearch(max_results=3)
tools = [search]
llm = ChatGroq(model="openai/gpt-oss-20b")
llm_with_tools = llm.bind_tools(tools)


tool_node = ToolNode(tools)

def chatbot(state: State):
  summary = state.get("summary")
  messages = None
  if summary:
    messages = [SystemMessage(content=f"## 이전 대화 요약:\n\n{summary}")] + state.get("messages")
  else:
    messages = state.get("messages")

  response = llm_with_tools.invoke(messages)
  return {"messages": [response]}


def summarize_conversation(state: State):
  previus_summary = state.get("summary")
  messages = None

  if previus_summary:
    messages = [SystemMessage(content=f"## 이전 대화 요약:\n\n{previus_summary}\n\n다음 대화 내용을 요약하여 기존 요약을 확장해주세요.")] + state.get("messages")
  else:
    messages = [SystemMessage(content="다음 대화 내용을 요약해주세요.")] + state.get("messages")

  new_summary = llm.invoke(messages).content
  new_messages = [RemoveMessage(id=x.id) for x in state.get("messages")[:-3]]
  return {"summary": new_summary, "messages": new_messages}


def summary_condition(state: State):
  messages = state.get("messages")
  last_message = messages[-1]

  if isinstance(last_message, AIMessage) and last_message.tool_calls:
    return tools_condition(state)

  if len(messages) > 6:
    return "summarize_conversation"

  return END


builder = StateGraph(State)
builder.add_node("chatbot", chatbot)
builder.add_node("tools", tool_node)
builder.add_node("summarize_conversation", summarize_conversation)

builder.add_conditional_edges("chatbot", summary_condition, {
    "tools": "tools",
    "summarize_conversation": "summarize_conversation",
    END: END
})
builder.add_edge("tools", "chatbot")
builder.set_entry_point("chatbot")
builder.set_finish_point("summarize_conversation")

memory = InMemorySaver()

graph = builder.compile(checkpointer=memory)

In [125]:
from IPython.display import display, Image, Markdown

# display(Markdown(graph.get_graph().draw_mermaid()))
# Image(graph.get_graph().draw_mermaid_png())
print(graph.get_graph().draw_mermaid())

---
config:
  flowchart:
    curve: linear
---
graph TD;
	__start__([<p>__start__</p>]):::first
	chatbot(chatbot)
	tools(tools)
	summarize_conversation(summarize_conversation)
	__end__([<p>__end__</p>]):::last
	__start__ --> chatbot;
	chatbot -.-> __end__;
	chatbot -.-> summarize_conversation;
	chatbot -.-> tools;
	tools --> chatbot;
	summarize_conversation --> __end__;
	classDef default fill:#f2f0ff,line-height:1.2
	classDef first fill-opacity:0
	classDef last fill:#bfb6fc



In [126]:
# summarize_conversation({"messages": [
#   HumanMessage(content="제 이름은 철수입니다."),
#   AIMessage(content="안녕하세요, 철수님! 저는 언제든지 도와드릴 준비가 되어 있습니다. 지금 궁금한 점이나 필요한 것이 있으면 말씀해 주세요!"),
# ]})

In [127]:
config = RunnableConfig(recursion_limit=10, configurable={"thread_id": 1})

for event in graph.stream({"messages": [HumanMessage("제 이름은 철수입니다.")]}, config):
  for value in event.values():
    if "messages" in value:
       value["messages"][-1].pretty_print()

================================== Ai Message ==================================

안녕하세요, 철수님! 무엇을 도와드릴까요?


In [128]:
for event in graph.stream({"messages": [HumanMessage("저는 AI 엔지니어입니다.")]}, config):
  for value in event.values():
    if "messages" in value:
       value["messages"][-1].pretty_print()

================================== Ai Message ==================================

안녕하세요, 철수님! AI 엔지니어이시라니 멋지네요. 현재 궁금하신 점이나 도움이 필요하신 부분이 있으신가요? 언제든 말씀해 주세요!


In [129]:
for event in graph.stream({"messages": [HumanMessage("취미는 독서와 영화 감상입니다.")]}, config):
  for value in event.values():
    if "messages" in value:
       value["messages"][-1].pretty_print()

================================== Ai Message ==================================

아, 철수님!  
AI 엔지니어이시고 독서와 영화 감상이 취미라니, 정말 흥미로운 조합이네요.  
혹시 최근에 읽은 책이나 감상한 영화 중에서 특히 인상 깊었던 것이 있나요?  
또는 AI 관련 프로젝트나 기술에 대해 이야기 나누고 싶으신 부분이 있으면 언제든 말씀해 주세요!


In [130]:
snapshot = graph.get_state(config)
snapshot


StateSnapshot(values={'messages': [HumanMessage(content='제 이름은 철수입니다.', additional_kwargs={}, response_metadata={}, id='52c0e55a-02d1-4512-82d5-6f7e9f99d5ca'), AIMessage(content='안녕하세요, 철수님! 무엇을 도와드릴까요?', additional_kwargs={'reasoning_content': 'The user says "제 이름은 철수입니다." meaning "My name is 철수" in Korean. Likely they want the assistant to address them by that name. No other request. So respond acknowledging name. Probably ask how can help.'}, response_metadata={'token_usage': {'completion_tokens': 75, 'prompt_tokens': 1363, 'total_tokens': 1438, 'completion_time': 0.075601307, 'prompt_time': 0.081812639, 'queue_time': 0.002765408, 'total_time': 0.157413946}, 'model_name': 'openai/gpt-oss-20b', 'system_fingerprint': 'fp_3023a70d60', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--90cbd904-c85d-4b0c-b42d-05e66616beec-0', usage_metadata={'input_tokens': 1363, 'output_tokens': 75, 'total_tokens': 1438}), HumanMessage(content='저는 AI 엔지니어입니다.', additional

In [131]:
for m in snapshot.values["messages"]:
  m.pretty_print()

================================ Human Message =================================

제 이름은 철수입니다.
================================== Ai Message ==================================

안녕하세요, 철수님! 무엇을 도와드릴까요?
================================ Human Message =================================

저는 AI 엔지니어입니다.
================================== Ai Message ==================================

안녕하세요, 철수님! AI 엔지니어이시라니 멋지네요. 현재 궁금하신 점이나 도움이 필요하신 부분이 있으신가요? 언제든 말씀해 주세요!
================================ Human Message =================================

취미는 독서와 영화 감상입니다.
================================== Ai Message ==================================

아, 철수님!  
AI 엔지니어이시고 독서와 영화 감상이 취미라니, 정말 흥미로운 조합이네요.  
혹시 최근에 읽은 책이나 감상한 영화 중에서 특히 인상 깊었던 것이 있나요?  
또는 AI 관련 프로젝트나 기술에 대해 이야기 나누고 싶으신 부분이 있으면 언제든 말씀해 주세요!


In [132]:
for event in graph.stream({"messages": [HumanMessage("제 이름과 직업, 그리고 취미를 기억하나요?")]}, config):
  for value in event.values():
    if "messages" in value:
       value["messages"][-1].pretty_print()

================================== Ai Message ==================================

네, 기억하고 있습니다!

- **이름**: 철수  
- **직업**: AI 엔지니어  
- **취미**: 독서와 영화 감상  

필요하신 내용이 더 있으면 언제든 말씀해 주세요!
================================ Remove Message ================================




In [133]:
snapshot = graph.get_state(config)
for m in snapshot.values["messages"]:
  m.pretty_print()
print("\n\nsummary:", snapshot.values["summary"])

================================== Ai Message ==================================

아, 철수님!  
AI 엔지니어이시고 독서와 영화 감상이 취미라니, 정말 흥미로운 조합이네요.  
혹시 최근에 읽은 책이나 감상한 영화 중에서 특히 인상 깊었던 것이 있나요?  
또는 AI 관련 프로젝트나 기술에 대해 이야기 나누고 싶으신 부분이 있으면 언제든 말씀해 주세요!
================================ Human Message =================================

제 이름과 직업, 그리고 취미를 기억하나요?
================================== Ai Message ==================================

네, 기억하고 있습니다!

- **이름**: 철수  
- **직업**: AI 엔지니어  
- **취미**: 독서와 영화 감상  

필요하신 내용이 더 있으면 언제든 말씀해 주세요!


summary: **대화 요약**

1. **사용자**  
   - 이름: 철수  
   - 직업: AI 엔지니어  
   - 취미: 독서와 영화 감상

2. **assistant**  
   - 사용자 정보를 기억하고 있다고 확인하고, 추가로 궁금한 점이 있으면 언제든 물어보라고 제안함.


In [134]:
for event in graph.stream({"messages": [HumanMessage("저는 최근에 귀멸의칼날 영화를 봤습니다.")]}, config):
  for value in event.values():
    if "messages" in value:
       value["messages"][-1].pretty_print()

================================== Ai Message ==================================

아, **귀멸의 칼날** 영화를 보셨군요!  
이 작품은 애니메이션 영화로 2022년에 개봉했으며, 원작 만화와 애니 시리즈의 이야기를 이어가면서도 새로운 캐릭터와 전투가 추가돼 많은 팬들 사이에서 큰 인기를 끌었죠.

**어떤 부분이 가장 기억에 남으셨나요?**  
- 전투 장면의 액션과 CG 효과  
- 주인공 탄지로와 가미코의 감정 변화  
- 음악과 사운드트랙  
- 혹은 다른 캐릭터(예: 무사시, 유우지 등)의 등장

또한, 영화에 이어 **시리즈**를 더 즐기고 싶으시다면  
- **시리즈 애니**: 1~3편까지 시청  
- **만화**: 원작 시리즈를 한 번 읽어보면 캐릭터와 배경이 더 풍부하게 느껴집니다.

혹시 영화에 대해 더 깊이 이야기 나누고 싶으신 점이나, 관련된 다른 작품 추천을 원하시면 언제든 말씀해 주세요!


In [135]:
for event in graph.stream({"messages": [HumanMessage("2025년 최근 개봉작입니다.")]}, config):
  for value in event.values():
    if "messages" in value:
       value["messages"][-1].pretty_print()

================================== Ai Message ==================================
Tool Calls:
  tavily_search (fc_688c20db-5092-4383-b9a9-2934f4f66b6d)
 Call ID: fc_688c20db-5092-4383-b9a9-2934f4f66b6d
  Args:
    end_date: 2025-12-31
    query: 귀멸의 칼날 2025 영화
    search_depth: advanced
    start_date: 2025-01-01
================================= Tool Message =================================
Name: tavily_search

{"query": "귀멸의 칼날 2025 영화", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://ko.wikipedia.org/wiki/%EA%B7%B9%EC%9E%A5%ED%8C%90_%EA%B7%80%EB%A9%B8%EC%9D%98_%EC%B9%BC%EB%82%A0:_%EB%AC%B4%ED%95%9C%EC%84%B1%ED%8E%B8", "title": "극장판 귀멸의 칼날: 무한성편 - 위키백과, 우리 모두의 백과사전", "content": "《극장판 귀멸의 칼날: 무한성편》(일본어: 劇場版「鬼滅の刃」無限城編영어: Demon Slayer: Infinity Castle)은 2025년 7월 18일에 공개된 일본의 애니메이션 영화로, 고토게 코요하루의 만화 《귀멸의 칼날》의 극장판이다. 원작 귀멸의 칼날의 최종결전 부분인 최종 국면 편의 3부작 중 첫번째를 다루고 있다.\n\n등장인물\n\n[편집]\n\nImage 12귀멸의 칼날의 등장인물 목록 문서를 참고하십시오.\n\n   CV 하나에 나츠키 - 카마도 탄지로\n   

In [136]:
snapshot = graph.get_state(config)
for m in snapshot.values["messages"]:
  m.pretty_print()
print("\n\nsummary:", snapshot.values["summary"])

================================== Ai Message ==================================
Tool Calls:
  tavily_search (fc_688c20db-5092-4383-b9a9-2934f4f66b6d)
 Call ID: fc_688c20db-5092-4383-b9a9-2934f4f66b6d
  Args:
    end_date: 2025-12-31
    query: 귀멸의 칼날 2025 영화
    search_depth: advanced
    start_date: 2025-01-01
================================= Tool Message =================================
Name: tavily_search

{"query": "귀멸의 칼날 2025 영화", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://ko.wikipedia.org/wiki/%EA%B7%B9%EC%9E%A5%ED%8C%90_%EA%B7%80%EB%A9%B8%EC%9D%98_%EC%B9%BC%EB%82%A0:_%EB%AC%B4%ED%95%9C%EC%84%B1%ED%8E%B8", "title": "극장판 귀멸의 칼날: 무한성편 - 위키백과, 우리 모두의 백과사전", "content": "《극장판 귀멸의 칼날: 무한성편》(일본어: 劇場版「鬼滅の刃」無限城編영어: Demon Slayer: Infinity Castle)은 2025년 7월 18일에 공개된 일본의 애니메이션 영화로, 고토게 코요하루의 만화 《귀멸의 칼날》의 극장판이다. 원작 귀멸의 칼날의 최종결전 부분인 최종 국면 편의 3부작 중 첫번째를 다루고 있다.\n\n등장인물\n\n[편집]\n\nImage 12귀멸의 칼날의 등장인물 목록 문서를 참고하십시오.\n\n   CV 하나에 나츠키 - 카마도 탄지로\n   

In [137]:
for event in graph.stream({"messages": [HumanMessage("비슷한 다른 영화를 추천해주실수 있나요?")]}, config):
  for value in event.values():
    if "messages" in value:
       value["messages"][-1].pretty_print()

================================== Ai Message ==================================

아래는 **“극장판 귀멸의 칼날: 무한성편”**과 분위기·주제·애니메이션 스타일이 비슷한, 2025년 이후 개봉 또는 2020년대에 인기를 끈 영화들입니다.  
각 작품의 핵심 포인트와 왜 비슷한지 짚어드릴게요.

| # | 영화 | 개봉 | 러닝타임 | 장르 | 비슷한 점 | 짧은 소개 |
|---|------|------|----------|------|-----------|-----------|
| 1 | **Jujutsu Kaisen 0** | 2021년 10월 (일본) | 108분 | 액션·판타지·스릴러 | **전투**와 **귀신·초자연적 존재**가 주인공을 둘러싸는 전개. | 주인공이 초자연적 힘을 활용해 악귀를 물리치는 스토리. |
| 2 | **Attack on Titan The Final Season (Part 1)** | 2023년 3월 (일본) | 130분 | 액션·판타지·전쟁 | **전투와 세계관의 대규모 전개**가 핵심. | 인간과 거대한 짐승(타이탄) 사이의 전쟁을 다룸. |
| 3 | **Fate/Stay Night: Heaven’s Feel (Part 3)** | 2020년 6월 (일본) | 120분 | 판타지·액션·드라마 | **전설과 전투**를 중심으로 한 스토리 라인. | 영웅들과 마법사가 대결하는 전투가 펼쳐짐. |
| 4 | **The Boy and the Heron** | 2023년 4월 (일본) | 140분 | 판타지·드라마 | **감성적**이면서도 **판타지 세계**가 결합. | 신비로운 생물과의 모험을 통해 성장하는 이야기. |
| 5 | **Your Name (Kimi no Na wa)** | 2016년 8월 (일본) | 106분 | 로맨스·판타지 | **시간/공간 이동**과 **감성적 연결**이 핵심. | 두 사람이 몸을 바꾸며 서로의 삶을 체험하는 감동 스토리. |


In [138]:
graph.get_state(config)

StateSnapshot(values={'messages': [AIMessage(content='', additional_kwargs={'reasoning_content': 'User says: "2025년 최근 개봉작입니다." They likely want info about a recent 2025 movie, possibly "귀멸의 칼날" but that is 2022. They might be mistaken or referring to a 2025 release? Actually, "귀멸의 칼날" movie released 2022. There might be a 2025 film "귀멸의 칼날 2"? Not sure. They might be asking for confirmation or details about 2025 release. We need to search for 2025 releases. Use search tool.', 'tool_calls': [{'id': 'fc_688c20db-5092-4383-b9a9-2934f4f66b6d', 'function': {'arguments': '{"end_date":"2025-12-31","query":"귀멸의 칼날 2025 영화","search_depth":"advanced","start_date":"2025-01-01"}', 'name': 'tavily_search'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 186, 'prompt_tokens': 1903, 'total_tokens': 2089, 'completion_time': 0.202967199, 'prompt_time': 0.023151899, 'queue_time': 0.003008961, 'total_time': 0.226119098, 'prompt_tokens_details': {'cached_tokens': 1536}}, '

In [139]:
snapshot = graph.get_state(config)
for m in snapshot.values["messages"]:
  m.pretty_print()
print("\n\nsummary:", snapshot.values["summary"])

================================== Ai Message ==================================
Tool Calls:
  tavily_search (fc_688c20db-5092-4383-b9a9-2934f4f66b6d)
 Call ID: fc_688c20db-5092-4383-b9a9-2934f4f66b6d
  Args:
    end_date: 2025-12-31
    query: 귀멸의 칼날 2025 영화
    search_depth: advanced
    start_date: 2025-01-01
================================= Tool Message =================================
Name: tavily_search

{"query": "귀멸의 칼날 2025 영화", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://ko.wikipedia.org/wiki/%EA%B7%B9%EC%9E%A5%ED%8C%90_%EA%B7%80%EB%A9%B8%EC%9D%98_%EC%B9%BC%EB%82%A0:_%EB%AC%B4%ED%95%9C%EC%84%B1%ED%8E%B8", "title": "극장판 귀멸의 칼날: 무한성편 - 위키백과, 우리 모두의 백과사전", "content": "《극장판 귀멸의 칼날: 무한성편》(일본어: 劇場版「鬼滅の刃」無限城編영어: Demon Slayer: Infinity Castle)은 2025년 7월 18일에 공개된 일본의 애니메이션 영화로, 고토게 코요하루의 만화 《귀멸의 칼날》의 극장판이다. 원작 귀멸의 칼날의 최종결전 부분인 최종 국면 편의 3부작 중 첫번째를 다루고 있다.\n\n등장인물\n\n[편집]\n\nImage 12귀멸의 칼날의 등장인물 목록 문서를 참고하십시오.\n\n   CV 하나에 나츠키 - 카마도 탄지로\n   